In [6]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
import dart_fss as dart
import json
import pymysql
import info

In [8]:
conn = pymysql.connect(host=info.myHost,user=info.myUser,password=info.myPw,db=info.myDb,charset="utf8")
cur = conn.cursor()

In [9]:
#삼성전자, LG화학, 카카오
company_code_list=["00126380","00356361","00258801"]
bsns_year=["2015","2016","2017","2018","2019","2020"]
#1분기보고서 : 11013, 반기보고서 : 110123, 
#분기보고서 : 11014, 사업보고서 : 11011
reprt_code=["11013","11012","11014","11011"]

In [10]:
sql ="truncate corp_financial_statement"
cur.execute(sql)

for year in bsns_year:
    for reprt in reprt_code:
        for company_code in company_code_list:
            url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key="+info.crtfc_key+"&corp_code="+company_code+"&bsns_year="+year+"&reprt_code="+reprt
            resultXML=urlopen(url)  #this is for response of XML
            stringText=resultXML.read().decode("utf-8")
            data = json.loads(stringText)
        
            #데이터가 접근가능상태인가?
            if(data["status"]=="000"):
                for item in data['list']:
                    #연결재무재표만 query
                    if item['fs_div'] in ['CFS']:   
                        if item['account_nm'] in ['매출액']:
                            ss = item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['영업이익']:
                            oi= item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['당기순이익']:
                            ni = item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['자본총계']:
                            tl = item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['부채총계']:
                            te = item['thstrm_amount'].replace(",","")
                
                #null check
                if ss:
                    if oi:
                        if ni:
                            if tl:
                                if te:
                                    sql = "INSERT INTO corp_financial_statement values ('"+company_code+"',"+year+",'"+reprt+"',"+ss+","+oi+","+ni+","+tl+","+te+")"
                                    ss, oi, ni, tl, te = None,None,None,None,None
                                    print(sql)
                                    cur.execute(sql)    
                    

INSERT INTO corp_financial_statement values ('00126380',2015,'11011',200653482000000,26413442000000,19060144000000,179059805000000,63119716000000)
INSERT INTO corp_financial_statement values ('00356361',2015,'11011',20206583000000,1823568000000,1148531000000,13103522000000,5475206000000)
INSERT INTO corp_financial_statement values ('00258801',2015,'11011',932151827086,88588297005,78764757714,2585474995627,603002737682)
INSERT INTO corp_financial_statement values ('00126380',2016,'11013',49782252000000,6675812000000,5252803000000,178326186000000,62913603000000)
INSERT INTO corp_financial_statement values ('00356361',2016,'11013',4874092000000,457745000000,338125000000,13106349000000,5753936000000)
INSERT INTO corp_financial_statement values ('00258801',2016,'11013',242483260813,21089734631,10949664522,3416726395085,1772972395012)
INSERT INTO corp_financial_statement values ('00126380',2016,'11012',50937120000000,8143950000000,5847393000000,181842516000000,62118157000000)
INSERT INTO cor

In [11]:
conn.commit()
conn.close()